In [61]:
import itertools
import plotly
import pandas as pd
import numpy as np
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

In [6]:
raw = pd.read_csv('./binned_metrics_cts.csv')

In [9]:
ct_drop = raw.drop('Unnamed: 0', axis=1)
ct_drop.head(1)

,start_censustract,distance_bins,duration,distance,velocity,BoroCode,mean_income,censustract,avg_travel_time,median_income,poverty_perc
0,1000201,"(0, 2]",10.314488,1.342711,0.130416,Manhattan,36150,1000201,31.536388,23036,44.8


In [30]:
# these are the columns to analyze
compare_list = ['duration', 'distance', 'velocity', 
                'mean_income', 'avg_travel_time', 
                'median_income', 'poverty_perc']
# build pairs of possible combinations
pairs = list(itertools.combinations(compare_list, 2))

# print out the pairs so you can delete the ones you don't want
for i, v in enumerate(pairs):
    print i, v

0 ('duration', 'distance')
1 ('duration', 'velocity')
2 ('duration', 'mean_income')
3 ('duration', 'avg_travel_time')
4 ('duration', 'median_income')
5 ('duration', 'poverty_perc')
6 ('distance', 'velocity')
7 ('distance', 'mean_income')
8 ('distance', 'avg_travel_time')
9 ('distance', 'median_income')
10 ('distance', 'poverty_perc')
11 ('velocity', 'mean_income')
12 ('velocity', 'avg_travel_time')
13 ('velocity', 'median_income')
14 ('velocity', 'poverty_perc')
15 ('mean_income', 'avg_travel_time')
16 ('mean_income', 'median_income')
17 ('mean_income', 'poverty_perc')
18 ('avg_travel_time', 'median_income')
19 ('avg_travel_time', 'poverty_perc')
20 ('median_income', 'poverty_perc')


In [58]:
# change the index of the delete func to remove the items we don't need to compare
ind2remove = [0, 1, 6, 12, 15, 16, 17, 18, 20]
finalpairs = [x for i,x in enumerate(pairs) if i not in ind2remove]

In [59]:
methods = ['pearson', 'spearman', 'kendall']
test_texts = []
dist_intervals = []
test_results = []
test_methods = []
def stat_analyzer(df, interval):
    for method in methods:
        for pair in finalpairs:
            test_text = pair[0] + ' and ' + pair[1]
            test_texts.append(test_text)
            dist_intervals.append(interval)
            test_results.append(df[pair[0]].corr(df[pair[1]], method=method))
            test_methods.append(method)
#             print method, "%s correlated to %s" % pair, df[pair[0]].corr(df[pair[1]], method=method)
        
dis_intervals = ct_drop.distance_bins.unique()
summary_df = pd.DataFrame(columns=['text', 'result'])
for interval in dis_intervals:
    interval_df = ct_drop[ct_drop['distance_bins']==interval]
    stat_analyzer(interval_df, interval)

test_results = pd.DataFrame({'test_method' : test_methods, 
                             'variables_tested' : test_texts, 
                             'dist_interval': dist_intervals, 
                             'test_result' : test_results})

test_results.head(1)

,dist_interval,test_method,test_result,variables_tested
0,"(0, 2]",pearson,-0.125734,duration and mean_income


In [60]:
print test_results.shape
test_results.sort_values('test_result')

(396, 4)


,dist_interval,test_method,test_result,variables_tested
144,"(16, 18]",pearson,-0.284381,duration and mean_income
108,"(14, 16]",pearson,-0.277711,duration and mean_income
180,"(18, 20]",pearson,-0.257236,duration and mean_income
72,"(12, 14]",pearson,-0.250115,duration and mean_income
112,"(14, 16]",pearson,-0.224172,distance and mean_income
36,"(10, 12]",pearson,-0.212828,duration and mean_income
114,"(14, 16]",pearson,-0.205137,distance and median_income
235,"(2, 4]",spearman,-0.198229,distance and poverty_perc
223,"(2, 4]",pearson,-0.191150,distance and poverty_perc
360,"(8, 10]",pearson,-0.182020,duration and mean_income
